In [ ]:
# Import libraries
!pip install -r requirements.txt
import numpy as np 
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt 
import pandas as pd

In [ ]:
# Import data
dataset = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.train.csv")

In [ ]:
# Replacement of empty spaces with null and elimination 
dataset['TotalCharges'] = dataset["TotalCharges"].replace(" ",np.nan)
dataset = dataset[dataset["TotalCharges"].notnull()]
dataset = dataset.reset_index()[dataset.columns]
# TotalCharges to float conversion
dataset["TotalCharges"] = dataset["TotalCharges"].astype(float)
# Substitution of variables to make them similar in the dataset
replace_nis = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection','TechSupport','StreamingTV', 'StreamingMovies']
for c in replace_nis : 
    dataset[c] = dataset[c].replace({'No internet service' : 'No'})
dataset["SeniorCitizen"] = dataset["SeniorCitizen"].replace({1:"Yes",0:"No"})

# Tenure division into categories
def tenure_div_cat(dataset) :
    if dataset["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (dataset["tenure"] > 12) & (dataset["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (dataset["tenure"] > 24) & (dataset["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (dataset["tenure"] > 48) & (dataset["tenure"] <= 60) :
        return "Tenure_48-60"
    elif dataset["tenure"] > 60 :
        return "Tenure_gt_60"
dataset["tenure_group"] = dataset.apply(lambda dataset:tenure_div_cat(dataset),axis = 1)

# Tenure column deletion, now divided into categories
dataset = dataset.drop(columns=['tenure'])

# CustomerID column deletion
dataset = dataset.drop(columns=['customerID'])

dataset.head()

In [ ]:
# Encoding dataset
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
for feature in dataset.columns:
    dataset[feature] = labelEncoder.fit_transform(dataset[feature])

In [ ]:
# Separate the variable (y) and all the features (X)
response = ["Churn"]
features = [i for i in dataset.columns if i not in response]
X = dataset[features]
y = dataset[response]

In [ ]:
# Division control
X.head()

In [ ]:
# Division control
y.head()

In [ ]:
# Creating training and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =.20 ,random_state=0)

In [ ]:
#Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix,accuracy_score

model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=False, 
                   intercept_scaling=1, class_weight='Balanced', random_state=None, solver='lbfgs', 
                   max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=10, l1_ratio=None)
model.fit(X_train,y_train.values.ravel())
y_pred = model.predict(X_test)

print("Using Logistic Regression we have an accuracy of:", accuracy_score(y_test,y_pred)*100, "%")
    
plot_confusion_matrix(model, X_test, y_test, cmap=plt.cm.YlGn, normalize='true' )
plt.show()